Group 2: 
Chris Sadlo, Glenda Decapia, Katrice Trahan, Sarah Kachelmeier

Project 2 - HURRICANES

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import country_converter as coco
from sqlalchemy import create_engine
import gmaps

# Import API key
from api_keys import weather_api_key
from api_keys import google_key
from config import username
from config import password


gmaps.configure(api_key = google_key)


# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

Store CSV files into dataframes

In [2]:
# EXTRACTING data from NOAA's HURDAT2 available at this website:
# https://www.kaggle.com/noaa/hurricane-database

csv_file = "Resources/atlantic.csv"
atlantic_data_df = pd.read_csv(csv_file)
atlantic_data_df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,HU,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,HU,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [3]:
# TRANSFORMING the Atlantic Ocean data by dropping columns that were largely unused

new_atlantic_data_df = atlantic_data_df[["ID","Name","Date","Time","Status","Latitude","Longitude","Maximum Wind","Minimum Pressure"]]
new_atlantic_data_df["Ocean"] = "ATLANTIC"

C:\Users\Mama\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
new_atlantic_data_df.head()

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,Ocean
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC


In [5]:
# EXTRACTING data from NOAA's HURDAT2 available at this website:
# https://www.kaggle.com/noaa/hurricane-database

csv_file = "Resources/pacific.csv"
pacific_data_df = pd.read_csv(csv_file)
pacific_data_df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,EP011949,UNNAMED,19490611,0,,TS,20.2N,106.3W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,EP011949,UNNAMED,19490611,600,,TS,20.2N,106.4W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,EP011949,UNNAMED,19490611,1200,,TS,20.2N,106.7W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,EP011949,UNNAMED,19490611,1800,,TS,20.3N,107.7W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,EP011949,UNNAMED,19490612,0,,TS,20.4N,108.6W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [6]:
# TRANSFORMING the Pacific Ocean data by dropping columns that were largely unused

new_pacific_data_df = pacific_data_df[["ID","Name","Date","Time","Status","Latitude","Longitude","Maximum Wind","Minimum Pressure"]]
new_pacific_data_df["Ocean"] = "PACIFIC"

C:\Users\Mama\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
# TRANSFORMING a combination of Atlantic Ocean and Pacific Ocean data into a single dataframe

atlantic_and_pacific_data = pd.concat([new_atlantic_data_df, new_pacific_data_df])
atlantic_and_pacific_data

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,Ocean
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC
...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,LO,21.7N,109.0W,35,1002,PACIFIC
26133,EP222015,SANDRA,20151128,1800,LO,22.4N,108.7W,30,1007,PACIFIC
26134,EP222015,SANDRA,20151129,0,LO,23.1N,108.3W,30,1008,PACIFIC
26135,EP222015,SANDRA,20151129,600,LO,23.5N,107.9W,25,1009,PACIFIC


Data Transformation

In [8]:
# CONFIRMING the datatypes that were Extracted

atlantic_and_pacific_data.dtypes

ID                  object
Name                object
Date                 int64
Time                 int64
Status              object
Latitude            object
Longitude           object
Maximum Wind         int64
Minimum Pressure     int64
Ocean               object
dtype: object

In [9]:
# TRANSFORMING by filtering the data from the CSV files to shrink the data to process

#################################
# With Filters (training brakes)
# Exclude any data from hurricane season prior to the year 2000  (Date >= year 2000)
#atlantic_and_pacific_2000 = atlantic_and_pacific_data.loc[atlantic_and_pacific_data["Date"] >= 20000000, :]

#clean_atlantic_and_pacific_df = atlantic_and_pacific_2000

# Exclude any data from hurricane season that was less than 60 mph (Maximum Wind >= 60)
#clean_atlantic_and_pacific_df = atlantic_and_pacific_2000.loc[atlantic_and_pacific_2000["Maximum Wind"] >= 100, :]
################################

################################
# Without training brakes
clean_atlantic_and_pacific_df = atlantic_and_pacific_data

clean_atlantic_and_pacific_df.shape

(75242, 10)

In [10]:
################################
# Adding a hurricane category

#def func(a_):
#    if a_ < 74:
#        return 0
#    elif a_ <= 95:
#        return 1
#    elif a_ <= 110:
#        return 2
#    elif a_ <= 129:
#        return 3
#    elif a_ <= 156:
#        return 4
#    else:
#        return 5
#    
#clean_atlantic_and_pacific_df['Category'] = clean_atlantic_and_pacific_df['Maximum Wind'].apply(lambda x: func(x[0]))
    
clean_atlantic_and_pacific_df['Category'] = 0

for index, row in clean_atlantic_and_pacific_df.iterrows():
    if row['Maximum Wind'] < 74:
        row['Category'] = 0
    elif row['Maximum Wind'] <= 95:
        row['Category'] = 1
    elif row['Maximum Wind'] <= 110:
        row['Category'] = 2
    elif row['Maximum Wind'] <= 129:
        row['Category'] = 3
    elif row['Maximum Wind'] <= 156:
        row['Category'] = 4
    else:
        row['Category'] = 5



In [11]:
# TRANSFORM the data by converting from a user-friendly format of latitude and longitude (eg. 51N, 101W)
# to one that was more commonly used with APIs (e.g. 51, -101)

# convert latitude and longitude values to decimal format
def cardinal2negative(s):
    degrees = float(s[:-1])
    cardinal = s[len(s)-1:]
    
    if cardinal in ('s','S','w','W'):
        degrees *= -1

    return degrees

# Perform this step to skip data copying warning from python
clean_atlantic_and_pacific_df = clean_atlantic_and_pacific_df.copy()
lat_series = clean_atlantic_and_pacific_df["Latitude"].copy().apply(cardinal2negative)
lng_series = clean_atlantic_and_pacific_df['Longitude'].copy().apply(cardinal2negative)

clean_atlantic_and_pacific_df["New Latitude"] = lat_series
clean_atlantic_and_pacific_df['New Longitude'] = lng_series

# TRANSFORM the data by removing the leading whitespace in-front of the
# hurricane's Name and Status left-over from CSV
clean_atlantic_and_pacific_df["Name"] = clean_atlantic_and_pacific_df["Name"].str.strip()
clean_atlantic_and_pacific_df["Status"] = clean_atlantic_and_pacific_df["Status"].str.strip()
clean_atlantic_and_pacific_df

clean_atlantic_and_pacific_df


,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,Ocean,Category,New Latitude,New Longitude
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC,0,28.0,-94.8
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC,0,28.0,-95.4
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC,0,28.0,-96.0
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC,0,28.1,-96.5
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC,0,28.2,-96.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,LO,21.7N,109.0W,35,1002,PACIFIC,0,21.7,-109.0
26133,EP222015,SANDRA,20151128,1800,LO,22.4N,108.7W,30,1007,PACIFIC,0,22.4,-108.7
26134,EP222015,SANDRA,20151129,0,LO,23.1N,108.3W,30,1008,PACIFIC,0,23.1,-108.3
26135,EP222015,SANDRA,20151129,600,LO,23.5N,107.9W,25,1009,PACIFIC,0,23.5,-107.9


In [12]:
##############################################################################
# THIS STEP WILL TAKE A FEW MINUTES TO RUN. PLEASE BE PATIENT (coco is slow) #
##############################################################################

# TRANSFORM the data by using the citipy API to determine the nearest city 
# for each lat lng pair and add it to the dataframe

unique_cities_list = []
city_list = []
country_list = []

# Create a set of latitude and longitude combinations to process over in a simple for-loop
lats = clean_atlantic_and_pacific_df["New Latitude"]
lngs = clean_atlantic_and_pacific_df["New Longitude"]
lat_lngs = zip(lats, lngs)

# EXTRACT from the citipy API to identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    
    # TRANSFORM the data by uppercasing cities and converting country codes to country names
    city = city.title()
    country = coco.convert(names=country, to='name_short')

    city_list.append(city)
    country_list.append(country)

    if (country == "not found"):
        continue ### We want to skip this location if we couldn't convert the country code

    city_country_pair = f"{city}, {country}"
    
    # If the city is unique, then add it to our unique cities list
    if city_country_pair not in unique_cities_list:
        unique_cities_list.append(city_country_pair)


# Add the city and country pair to the dataframe here:
clean_atlantic_and_pacific_df["City"] = city_list
clean_atlantic_and_pacific_df["Country"] = country_list

### Warning for the Netherland's Antilles island's country code "AN" for not being in citipy

In [13]:
#clean_atlantic_and_pacific_df.head()

In [14]:
# TRANSFORM the data by filtering out any location where the country code could not be converted
clean_atlantic_and_pacific_df = clean_atlantic_and_pacific_df[clean_atlantic_and_pacific_df["Country"] != "not found"]

# Print the city count to confirm sufficient count
len(unique_cities_list)
unique_cities_list

['Freeport, United States',
 'Port Lavaca, United States',
 'Rockport, United States',
 'Beeville, United States',
 'Pleasanton, United States',
 'Hondo, United States',
 'Uvalde, United States',
 'San Angelo, United States',
 'Miramar, Mexico',
 'Scarborough, Trinidad and Tobago',
 'Sinnamary, French Guiana',
 'Bathsheba, Barbados',
 'Le Vauclin, Martinique',
 'Saint-Francois, Guadeloupe',
 'Charlestown, St. Kitts and Nevis',
 'Arroyo, United States',
 'Cabo Rojo, United States',
 'San Pedro De Macoris, Dominican Republic',
 'Tamayo, Dominican Republic',
 'Verrettes, Haiti',
 'Jeremie, Haiti',
 'Santiago De Cuba, Cuba',
 'Amancio, Cuba',
 'Venezuela, Cuba',
 'Cienfuegos, Cuba',
 'Pedro Betancourt, Cuba',
 'Bauta, Cuba',
 'Bahia Honda, Cuba',
 'Venice, United States',
 'Saint Pete Beach, United States',
 'Panama City, United States',
 'Destin, United States',
 'Bainbridge, United States',
 'Albany, United States',
 'Dublin, United States',
 'Aiken, United States',
 'Florence, United St

In [15]:
#clean_atlantic_and_pacific_df.head()

In [16]:
clean_atlantic_and_pacific_df.shape

(74996, 15)

In [17]:
# LOAD - Perform the loading of the data into the sql server here:
rds_connection_string = f'{username}:{password}@localhost:5432/hurricanes'
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm the table name in the database:
engine.table_names()


['hurricanes']

In [18]:
column_names = clean_atlantic_and_pacific_df.columns
column_names

Index(['ID', 'Name', 'Date', 'Time', 'Status', 'Latitude', 'Longitude',
       'Maximum Wind', 'Minimum Pressure', 'Ocean', 'Category', 'New Latitude',
       'New Longitude', 'City', 'Country'],
      dtype='object')

In [19]:
clean_atlantic_and_pacific_df.loc[clean_atlantic_and_pacific_df['Maximum Wind'] < 74, 'Category'] = 0
clean_atlantic_and_pacific_df.loc[ (clean_atlantic_and_pacific_df['Maximum Wind'] >= 74) & (clean_atlantic_and_pacific_df['Maximum Wind'] < 96) , 'Category'] = 1
clean_atlantic_and_pacific_df.loc[ (clean_atlantic_and_pacific_df['Maximum Wind'] >= 96) & (clean_atlantic_and_pacific_df['Maximum Wind'] < 111) , 'Category'] = 2
clean_atlantic_and_pacific_df.loc[ (clean_atlantic_and_pacific_df['Maximum Wind'] >= 111) & (clean_atlantic_and_pacific_df['Maximum Wind'] < 130) , 'Category'] = 3
clean_atlantic_and_pacific_df.loc[ (clean_atlantic_and_pacific_df['Maximum Wind'] >= 130) & (clean_atlantic_and_pacific_df['Maximum Wind'] < 157) , 'Category'] = 4
clean_atlantic_and_pacific_df.loc[ (clean_atlantic_and_pacific_df['Maximum Wind'] >= 157) , 'Category'] = 5

C:\Users\Mama\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [20]:
clean_atlantic_and_pacific_df.head(50)

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,Ocean,Category,New Latitude,New Longitude,City,Country
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC,1,28.0,-94.8,Freeport,United States
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC,1,28.0,-95.4,Freeport,United States
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC,1,28.0,-96.0,Port Lavaca,United States
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC,1,28.1,-96.5,Port Lavaca,United States
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC,1,28.2,-96.8,Rockport,United States
5,AL011851,UNNAMED,18510626,0,HU,28.2N,97.0W,70,-999,ATLANTIC,0,28.2,-97.0,Rockport,United States
6,AL011851,UNNAMED,18510626,600,TS,28.3N,97.6W,60,-999,ATLANTIC,0,28.3,-97.6,Beeville,United States
7,AL011851,UNNAMED,18510626,1200,TS,28.4N,98.3W,60,-999,ATLANTIC,0,28.4,-98.3,Beeville,United States
8,AL011851,UNNAMED,18510626,1800,TS,28.6N,98.9W,50,-999,ATLANTIC,0,28.6,-98.9,Pleasanton,United States
9,AL011851,UNNAMED,18510627,0,TS,29.0N,99.4W,50,-999,ATLANTIC,0,29.0,-99.4,Hondo,United States


In [21]:
clean_atlantic_and_pacific_df.dtypes

ID                   object
Name                 object
Date                  int64
Time                  int64
Status               object
Latitude             object
Longitude            object
Maximum Wind          int64
Minimum Pressure      int64
Ocean                object
Category              int64
New Latitude        float64
New Longitude       float64
City                 object
Country              object
dtype: object

In [22]:
# TRANSFORM the data by converting the column names to something more palatable by postgres

# Construct an empty list to be populated soon
sql_column_names = []

# Make a deep copy of the hurricane dataframe
to_sql_df = clean_atlantic_and_pacific_df.copy()

# Loop over all the column names
for name in column_names:
    
    # And append them to a list of lowercased column names
    sql_column_names.append(name.lower())
    
# Set all the column names in the deep-copy dataframe to their lowercase equivalents
to_sql_df.columns = sql_column_names

# TRANSFORM
# Rename the columns that have spaces in the middle to use underscores instead
to_sql_df.rename(columns={"date":"date_stamp", 
                          "time":"time_stamp", 
                          "maximum wind":"max_wind",
                          "minimum pressure":"min_pressure",
                          "new latitude":"new_latitude",
                          "new longitude":"new_longitude",
                          "ocean": "ocean",
                          "category": "category"}, inplace=True)

to_sql_df

,id,name,date_stamp,time_stamp,status,latitude,longitude,max_wind,min_pressure,ocean,category,new_latitude,new_longitude,city,country
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC,1,28.0,-94.8,Freeport,United States
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC,1,28.0,-95.4,Freeport,United States
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC,1,28.0,-96.0,Port Lavaca,United States
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC,1,28.1,-96.5,Port Lavaca,United States
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC,1,28.2,-96.8,Rockport,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,LO,21.7N,109.0W,35,1002,PACIFIC,0,21.7,-109.0,Cabo San Lucas,Mexico
26133,EP222015,SANDRA,20151128,1800,LO,22.4N,108.7W,30,1007,PACIFIC,0,22.4,-108.7,Cabo San Lucas,Mexico
26134,EP222015,SANDRA,20151129,0,LO,23.1N,108.3W,30,1008,PACIFIC,0,23.1,-108.3,Eldorado,Mexico
26135,EP222015,SANDRA,20151129,600,LO,23.5N,107.9W,25,1009,PACIFIC,0,23.5,-107.9,Eldorado,Mexico


In [23]:
to_sql_df.shape

(74996, 15)

In [24]:
to_sql_df.rename(columns = {'max_wind': 'wind'}, inplace = True)
to_sql_df.head()

,id,name,date_stamp,time_stamp,status,latitude,longitude,wind,min_pressure,ocean,category,new_latitude,new_longitude,city,country
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC,1,28.0,-94.8,Freeport,United States
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC,1,28.0,-95.4,Freeport,United States
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC,1,28.0,-96.0,Port Lavaca,United States
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC,1,28.1,-96.5,Port Lavaca,United States
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC,1,28.2,-96.8,Rockport,United States


In [25]:
# Store a backup copy of the dataframe so that we don't have to rerun the above.
# This will allow others to quickly spin up the full-size database
to_sql_df.to_csv("Resources/final_database.csv", index=True, header=True)

In [26]:
csv_file = "Resources/final_database.csv"
recovered_df = pd.read_csv(csv_file)
to_sql_df = recovered_df
recovered_df.head(50)

,Unnamed: 0,id,name,date_stamp,time_stamp,status,latitude,longitude,wind,min_pressure,ocean,category,new_latitude,new_longitude,city,country
0,0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999,ATLANTIC,1,28.0,-94.8,Freeport,United States
1,1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999,ATLANTIC,1,28.0,-95.4,Freeport,United States
2,2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999,ATLANTIC,1,28.0,-96.0,Port Lavaca,United States
3,3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999,ATLANTIC,1,28.1,-96.5,Port Lavaca,United States
4,4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999,ATLANTIC,1,28.2,-96.8,Rockport,United States
5,5,AL011851,UNNAMED,18510626,0,HU,28.2N,97.0W,70,-999,ATLANTIC,0,28.2,-97.0,Rockport,United States
6,6,AL011851,UNNAMED,18510626,600,TS,28.3N,97.6W,60,-999,ATLANTIC,0,28.3,-97.6,Beeville,United States
7,7,AL011851,UNNAMED,18510626,1200,TS,28.4N,98.3W,60,-999,ATLANTIC,0,28.4,-98.3,Beeville,United States
8,8,AL011851,UNNAMED,18510626,1800,TS,28.6N,98.9W,50,-999,ATLANTIC,0,28.6,-98.9,Pleasanton,United States
9,9,AL011851,UNNAMED,18510627,0,TS,29.0N,99.4W,50,-999,ATLANTIC,0,29.0,-99.4,Hondo,United States


In [27]:
#############################################

In [28]:
# LOAD the data INTO the SQL server database
to_sql_df.to_sql(name='hurricanes', con=engine, if_exists='replace', index=True)

In [29]:
# LOAD/EXTRACT the data FROM the SQL database
katrina_from_sql_df = pd.read_sql_query("select * from hurricanes where name = 'KATRINA'", con=engine)
coords = katrina_from_sql_df[["new_latitude", "new_longitude"]]

# EXTRACT Google Maps data
fig = gmaps.figure(zoom_level=5, center=(27.5,-85))

#heat_layer = gmaps.heatmap_layer(coords, dissipating=False, max_intensity=100, point_radius=3)
#fig.add_layer(heat_layer)

# TRANSFORM the EXTRACTED data into a Google Maps symbol layer
hurricane_katrina_layer = gmaps.symbol_layer(
    coords, fill_color='red', stroke_color='red', scale=2
)
fig.add_layer(hurricane_katrina_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [31]:
# LOAD/EXTRACT the data FROM the SQL database
all_hurricanes_from_sql_df = pd.read_sql_query('select * from hurricanes where wind >= 115', con=engine)
coords = all_hurricanes_from_sql_df[["new_latitude", "new_longitude"]]

# EXTRACT Google Maps data
fig = gmaps.figure(zoom_level=3, center=(25,-75))

# TRANSFORM the EXTRACTED data into a Google Maps symbol layer
all_hurricanes_layer = gmaps.symbol_layer(
    coords, fill_color='green', stroke_color='green', scale=2
)
fig.add_layer(all_hurricanes_layer)
fig.add_layer(hurricane_katrina_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# LOAD/EXTRACT the data FROM the SQL database
all_hurricanes_from_sql_df = pd.read_sql_query('select * from hurricanes where wind >= 115', con=engine)
coords = all_hurricanes_from_sql_df[["new_latitude", "new_longitude"]]
winds = all_hurricanes_from_sql_df["wind"]

winds.describe()

count    1836.000000
mean      123.739107
std         9.832550
min       115.000000
25%       115.000000
50%       120.000000
75%       130.000000
max       185.000000
Name: wind, dtype: float64

In [34]:
# EXTRACT Google Maps data
fig = gmaps.figure(zoom_level=3, center=(25,-75))

# TRANSFORM the EXTRACTED data into a Google Maps heatmap layer
heat_layer = gmaps.heatmap_layer(coords, weights=winds, dissipating=True, max_intensity=150, point_radius=7)

fig.add_layer(heat_layer)
fig.add_layer(hurricane_katrina_layer)
fig

Figure(layout=FigureLayout(height='420px'))